## Training

![](images/ml-artifacts-1.jpg)

In [1]:
!rm -fr train
!mkdir train

In [2]:
import warnings
import pandas as pd
import tensorflow as tf

warnings.filterwarnings('ignore')

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

BUCKET = 'train/'

# Extraction process
dataset = pd.read_csv('https://storage.googleapis.com/jchavezar-public-datasets/auto-mpg.csv')
dataset.tail()

dataset.isna().sum()
dataset = dataset.dropna()
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()

train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')


def norm(x):
    return (x - train_stats['mean']) / train_stats['std']


normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)


def build_model():
    model_ai = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model_ai.compile(loss='mse',
                     optimizer=optimizer,
                     metrics=['mae', 'mse'])
    return model_ai


model = build_model()
model.summary()
EPOCHS = 1000

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
early_history = model.fit(normed_train_data, train_labels,
                          epochs=EPOCHS, validation_split=0.2,
                          callbacks=[early_stop])

# Export model and save to GCS
print(BUCKET)
model.save(BUCKET)

2023-02-03 18:22:21.111108: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.11.0
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                640       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000


2023-02-03 18:22:24.116057: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


8/8 [==============================] - 1s 42ms/step - loss: 564.8606 - mae: 22.5435 - mse: 564.8606 - val_loss: 553.4888 - val_mae: 22.3036 - val_mse: 553.4888
Epoch 2/1000
8/8 [==============================] - 0s 5ms/step - loss: 513.1585 - mae: 21.4281 - mse: 513.1585 - val_loss: 499.9730 - val_mae: 21.1418 - val_mse: 499.9730
Epoch 3/1000
8/8 [==============================] - 0s 4ms/step - loss: 462.6396 - mae: 20.3071 - mse: 462.6396 - val_loss: 443.6070 - val_mae: 19.8434 - val_mse: 443.6070
Epoch 4/1000
8/8 [==============================] - 0s 4ms/step - loss: 408.4442 - mae: 19.0185 - mse: 408.4442 - val_loss: 381.9565 - val_mae: 18.3183 - val_mse: 381.9565
Epoch 5/1000
8/8 [==============================] - 0s 4ms/step - loss: 350.4197 - mae: 17.5306 - mse: 350.4197 - val_loss: 318.0355 - val_mae: 16.5966 - val_mse: 318.0355
Epoch 6/1000
8/8 [==============================] - 0s 4ms/step - loss: 291.3926 - mae: 15.8563 - mse: 291.3926 - val_loss: 255.4471 - val_mae: 14.7068 

INFO:tensorflow:Assets written to: train/assets


INFO:tensorflow:Assets written to: train/assets


## Create Container Image and Save it on Google Registry

![](images/ml-artifacts-2.jpg)

In [3]:
!rm -fr app
!mkdir app

In [4]:
!pwd

/rapids/notebooks/vertex-ai-mlops/from_local_to_production


In [5]:
%%writefile app/main.py
from fastapi import Request, FastAPI
from tensorflow import keras
import json
import os

app = FastAPI()

model = keras.models.load_model('.')

@app.get('/')
def get_root():
    return {'message': 'Welcome mpg API: miles per gallon prediction'}


@app.get('/health_check')
def health():
    return 200


if os.environ.get('AIP_PREDICT_ROUTE') is not None:
    method = os.environ['AIP_PREDICT_ROUTE']
else:
    method = '/predict'

print(method)
@app.post(method)
async def predict(request: Request):
    print("----------------- PREDICTING -----------------")
    body = await request.json()
    instances = body["instances"]
    outputs = model.predict(instances)
    response = outputs.tolist()
    print("----------------- OUTPUTS -----------------")

    return {"predictions": response}

Writing app/main.py


In [6]:
%%writefile Dockerfile
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY train /app
COPY app /app
WORKDIR /app
RUN pip install sklearn joblib pandas tensorflow
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]

EXPOSE 8080

Writing Dockerfile


In [8]:
!gcloud builds submit -t gcr.io/jchavezar-demo/custom-local-train:v1 .

Creating temporary tarball archive of 11 file(s) totalling 549.0 KiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/root/.config/gcloud/logs/2023.02.03/18.38.52.481156.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://jchavezar-demo_cloudbuild/source/1675449532.587196-1333156387d849c8aca34a55f8631671.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jchavezar-demo/locations/global/builds/aa47cc52-1390-4263-a5bb-27d3fdd70158].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/aa47cc52-1390-4263-a5bb-27d3fdd70158?project=569083142710 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "aa47cc52-1390-4263-a5bb-27d3fdd70158"

FETCHSOURCE
Fetching storage object: gs://jchavezar-demo_cloudbuild/source/1675449532.587196-1333156387d849c8aca34a55f863167

## Upload Image to Model Registry

In [1]:
from google.cloud import aiplatform as aip

my_model = aip.Model.upload(
    display_name="my-model-latest", 
    serving_container_ports=[8080],
    serving_container_health_route="/health_check",
    serving_container_image_uri="gcr.io/jchavezar-demo/custom-local-train:v1")

Creating Model
Create Model backing LRO: projects/569083142710/locations/us-central1/models/6666613877112832000/operations/3402666124901351424
Model created. Resource name: projects/569083142710/locations/us-central1/models/6666613877112832000@1
To use this Model in another session:
model = aiplatform.Model('projects/569083142710/locations/us-central1/models/6666613877112832000@1')


## Deploy on Vertex Endpoints

![](images/ml-artifacts-3.jpg)

In [ ]:
my_model.deploy(
    deployed_model_display_name='local-model',
    machine_type='n1-standard-4'
)

Creating Endpoint
Create Endpoint backing LRO: projects/569083142710/locations/us-central1/endpoints/6255134844557197312/operations/6690293852881813504
Endpoint created. Resource name: projects/569083142710/locations/us-central1/endpoints/6255134844557197312
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/569083142710/locations/us-central1/endpoints/6255134844557197312')
Deploying model to Endpoint : projects/569083142710/locations/us-central1/endpoints/6255134844557197312
Deploy Endpoint model backing LRO: projects/569083142710/locations/us-central1/endpoints/6255134844557197312/operations/2249744620294504448
